# 時系列リターン評価の実行例

In [1]:
import numpy as np
import pandas as pd
from utils.paths import Paths

trade_history = pd.read_csv(Paths.TRADE_HISTORY_CSV)
trade_history.tail(2)

,日付,売or買,業種,銘柄コード,社名,株数,取得単価,決済単価,取得価格,決済価格,手数料,利益（税引前）,利率（税引前）
14089,2025-06-24,買,建設（中堅ゼネコン・建材）,5911,横河ブリッジホールディングス,200,2549.0,2531.0,509800,506200,0,-3600,-0.007062
14090,2025-06-24,買,建設（中堅ゼネコン・建材）,6737,ＥＩＺＯ,200,2115.0,2114.0,423000,422800,0,-200,-0.000473


In [2]:
from return_metrics_tool import (
    DailyReturnGenerator,
    ReturnMetricsRunner,
    MetricsInteractiveViewer,
)

drg = DailyReturnGenerator(date_series=trade_history["日付"],
                     acquisition_price_series=trade_history["取得価格"],
                     settlement_price_series=trade_history["決済価格"],
                     long_or_short_series=trade_history["売or買"],
                     short_keyphrase="売")
daily_return_df = drg.get()


runner = ReturnMetricsRunner(date_series=daily_return_df.index,
                             return_series=daily_return_df['Return'])
metrics = runner.calculate()
viewer = MetricsInteractiveViewer(metrics)
viewer.display()

Output()